In [1]:
import torch
from PIL import Image
import requests
import torchvision.transforms as T
import matplotlib.pyplot as plt
from collections import defaultdict
import torch.nn.functional as F
import numpy as np
from skimage.measure import find_contours

from matplotlib import patches,  lines
from matplotlib.patches import Polygon

torch.set_grad_enabled(False);

In [2]:
%matplotlib notebook
%matplotlib inline

VQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/VQA'
GQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/GQA'

import pandas as pd
import json
import pickle as pkl
import numpy as np
import h5py
import random
from PIL import Image
import IPython.display as display
from collections import defaultdict
import os

import matplotlib.pyplot as plt
import torch

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [50]:
import clip

CUDA_DEVICE=1
torch.cuda.set_device(CUDA_DEVICE)
device = torch.device("cuda:"+str(CUDA_DEVICE))
#the default ipykernel links to the first conda environment

model, postprocessor = torch.hub.load('ashkamath/mdetr:main', 'mdetr_efficientnetB5', pretrained=True, return_postprocessor=True)
model = model.to(device)
model.eval();

clip_model, preprocess = clip.load("ViT-B/32",device=device)

Using cache found in /root/.cache/torch/hub/ashkamath_mdetr_main
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
val_all=load_json(os.path.join(VQA_PATH,'scenes','val_all.json'))
print ('Length of testing data:',len(val_all))

Length of testing data: 214354


In [14]:
q_type_counter=defaultdict(int)

number_q={}
for i,row in enumerate(val_all):
    if row['ans_type']!='number':
        continue
    number_q[row['idx']]={
        'question':row['question'],
        'img_id':row['img_id'],
        'answer':row['answer'],
        'scores':row['scores'],
        'q_type':row['q_type'],
        'ans_type':row['ans_type']
    }    
    q_type_counter[row['q_type']]+=1
print (q_type_counter)
print (len(number_q))

defaultdict(<class 'int'>, {'how many': 20320, 'how many people are': 1997, 'what': 840, 'what time': 1130, 'what number is': 656, 'how': 877, 'none of the above': 260, 'how many people are in': 900, 'what is': 45, 'what are the': 77, 'what is the': 757, 'is it': 3, 'which': 78, 'is the': 7, 'what does the': 86, 'does the': 8, 'what is this': 7, 'what is the name': 14, 'why': 5, 'who is': 3, 'what animal is': 2, 'can you': 2, 'is this a': 1, 'what brand': 2, 'what is on the': 4, 'is he': 2, 'is this person': 3, 'is this': 9, 'where is the': 5, 'are these': 2, 'is the man': 2, 'what type of': 4, 'is the woman': 1, 'are the': 2, 'is': 3, 'is that a': 2, 'are there': 3, 'is there': 4, 'what color is the': 1, 'was': 1, 'what kind of': 4, 'what are': 2, 'what is the man': 3})
28134


In [8]:
names=list(number_q.keys())
print (len(names))

28134


In [19]:
counting_ent=load_pkl(os.path.join(VQA_PATH,'parser-related','counting_related.pkl'))
print (len(counting_ent))

23372


In [37]:
number_temp={}
with open('../clip_vqa/number_prompt.txt') as number:
    for line in number:
        line = line.strip().split('\t')
        qid, quest, prompt = line[0], line[1].lower(), json.loads(line[2])
        number_temp[qid]=prompt

In [78]:
print (len(number_temp))

4932


In [46]:
vis=0
random.shuffle(names)
for name in names:
    if vis>10:
        break
    info=number_q[name]
    if info['q_type'] not in ['what number is']:
        continue
    print ('\n',info['question'])
    print ('\t',info['answer'],info['q_type'])
    print ('\t',number_temp[name][0])
    vis+=1


 What number is on the black jersey?
	 ['33'] what number is
	 number [mask] is on the black jersey.

 What number is on the train car?
	 ['9018', '9048'] what number is
	 number [mask] is on the train car.

 What number is on the man's shirt?
	 ['26'] what number is
	 number [mask] is on the man's shirt.

 What number is on her jersey?
	 ['42'] what number is
	 number [mask] is on her jersey.

 What number is on the batter's uniform?
	 ['10', '12', '11', '18'] what number is
	 [mask] is on the batter's uniform.

 What number is at the front of the motorcycle?
	 ['213'] what number is
	 number [mask] is at the front of the motorcycle.

 What number is on the train?
	 ['5867', '4'] what number is
	 number [mask] is on the train.

 What number is bus?
	 ['18'] what number is
	 number [mask] is bus.

 What number is the fielder?
	 ['44'] what number is
	 number [mask] is the fielder.

 What number is on the fence?
	 ['4'] what number is
	 number [mask] is on the fence.

 What number is o

In [67]:
time_candidates=['morning','night','noon','afternoon']
number_mapper={'zero': '0',
               'one': '1',
               'two': '2',
               'three': '3',
               'four': '4',
               'five': '5',
               'six': '6',
               'seven': '7',
               'eight': '8',
               'nine': '9',
               'ten': '10',
               'eleven':'11',
               'twelve':'12'}
number_candidates=[n for n in number_mapper]

In [20]:
import inflect
stemmer = inflect.engine()

In [26]:
def pred_with_scenic_det(name,thresh):
    result=load_pkl(os.path.join(VQA_PATH,
                                 'features/one-all-scenic',
                                 str(name)+'.pkl'))
    ent=counting_ent[name]['ent']
    obj=stemmer.singular_noun(ent)
    if obj==False:
        obj=ent
    counter=0
    for info in result[obj]:
        if info['score']>=thresh:
            counter+=1
    return counter

In [ ]:
def pred_with_mdert(cap,img_id):

In [69]:
def pred_counting(name,ques,q_t,img_id,thresh=0.2):
    if q_t in ['how many','how many people are','how many people are in']:
        if q_t in ['how many people are','how many people are in']:
            thresh=0.13
        else:
            thresh=0.2
        pred=pred_with_scenic_det(name,thresh)
    else:
        if q_t in ['what time is','what time']:
            candidates=time_candidates
        else:
            candidates=number_candidates
        img_id=str(row['img_id'])
        #print (q_t,candidates)
        img_id='COCO_val2014_'+img_id.zfill(12)+'.jpg'
        prompt=number_temp[name][0]
        pred=pred_with_statement(img_id,prompt,candidates)
        if q_t not in ['what time is','what time']:
            pred=number_mapper[pred]
    """
    elif q_t in ['what number is']:
        img_id=str(row['img_id'])
        img_id='COCO_val2014_'+img_id.zfill(12)+'.jpg'
        cap=ques.lower()[:-1].replace(q_t,'')
        pred=pred_with_mdert(cap,img_id)
    """

    pred=str(pred)
    return pred

In [52]:
def pred_with_statement(img_id,prompts,candidates):
    #tokens=clip.tokenize(candidates)
    img_feat=Image.open(os.path.join(VQA_PATH,'images/val2014',img_id))
    sents=[prompts.replace('[mask]',c) for c in candidates]
    with torch.no_grad():
        image_features = clip_model.encode_image(preprocess(img_feat).unsqueeze(0).to(device))
        text_features = clip_model.encode_text(clip.tokenize(sents).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity.topk(2)
    #print (indices[0][0].item())
    ans=candidates[indices[0][0].item()]
    return ans

In [77]:
vis=0
#random.shuffle(names)
invalid=[]
acc={q:0.0 for q in q_type_counter}
counter={q:0.0 for q in q_type_counter}
for name in names:
    #if vis>100:
    #    break
    #if name in counting_ent:
    #    continue
    info=number_q[name]
    #print ('\n',info['question'])
    #print ('\t',opt_ent[name]['ent'])
    try:
        pred=pred_counting(name,info['question'],info['q_type'],info['img_id'])
    except:
        invalid.append(name)
    #print ('\n',pred,info['answer'])
    vis+=1
    counter[info['q_type']]+=1
    if pred in info['answer']:
        score=info['scores'][info['answer'].index(pred)]
        acc[info['q_type']]+=score
        
    if vis%500==0:
        print('Already finish:',vis,len(invalid))
        for q_t in counter:
            if counter[q_t]==0:
                continue
            print ('\tQuestion type:%s, Accuracy %.2f' % 
                   (q_t,acc[q_t]*100.0/counter[q_t]))
        print('\tAccuracy %.2f\n' % 
              (sum(acc.values())*100.0/sum(counter.values())))  

print ('Overall')
for q_t in counter:
    if counter[q_t]==0:
        continue
    print ('\tQuestion type:%s, Accuracy %.2f, Number of instance %d' % 
           (q_t,acc[q_t]*100.0/counter[q_t],counter[q_t]))
print('\tAccuracy %.2f' % 
      (sum(acc.values())*100.0/sum(counter.values())))  
print ('Invalid',len(invalid))

Already finish: 500 1
	Question type:how many, Accuracy 37.59
	Question type:how many people are, Accuracy 37.65
	Question type:what, Accuracy 11.58
	Question type:what time, Accuracy 0.00
	Question type:what number is, Accuracy 0.00
	Question type:how, Accuracy 5.71
	Question type:none of the above, Accuracy 0.00
	Question type:how many people are in, Accuracy 60.95
	Question type:what is the, Accuracy 0.00
	Question type:which, Accuracy 0.00
	Question type:what does the, Accuracy 0.00
	Accuracy 32.04

Already finish: 1000 3
	Question type:how many, Accuracy 38.18
	Question type:how many people are, Accuracy 37.29
	Question type:what, Accuracy 10.86
	Question type:what time, Accuracy 0.86
	Question type:what number is, Accuracy 4.80
	Question type:how, Accuracy 3.00
	Question type:none of the above, Accuracy 0.00
	Question type:how many people are in, Accuracy 69.46
	Question type:what are the, Accuracy 0.00
	Question type:what is the, Accuracy 0.00
	Question type:which, Accuracy 25.0

Already finish: 5500 8
	Question type:how many, Accuracy 38.09
	Question type:how many people are, Accuracy 42.26
	Question type:what, Accuracy 7.73
	Question type:what time, Accuracy 0.70
	Question type:what number is, Accuracy 2.69
	Question type:how, Accuracy 3.81
	Question type:none of the above, Accuracy 2.55
	Question type:how many people are in, Accuracy 68.33
	Question type:what is, Accuracy 0.00
	Question type:what are the, Accuracy 0.00
	Question type:what is the, Accuracy 3.53
	Question type:is it, Accuracy 0.00
	Question type:which, Accuracy 10.67
	Question type:is the, Accuracy 0.00
	Question type:what does the, Accuracy 0.00
	Question type:does the, Accuracy 0.00
	Question type:what is the name, Accuracy 0.00
	Question type:why, Accuracy 0.00
	Question type:who is, Accuracy 0.00
	Question type:is he, Accuracy 0.00
	Question type:is this person, Accuracy 0.00
	Question type:is this, Accuracy 0.00
	Question type:where is the, Accuracy 0.00
	Question type:are these, Accuracy

Already finish: 9500 16
	Question type:how many, Accuracy 37.86
	Question type:how many people are, Accuracy 45.15
	Question type:what, Accuracy 6.22
	Question type:what time, Accuracy 1.18
	Question type:what number is, Accuracy 5.47
	Question type:how, Accuracy 3.07
	Question type:none of the above, Accuracy 7.73
	Question type:how many people are in, Accuracy 66.90
	Question type:what is, Accuracy 0.00
	Question type:what are the, Accuracy 0.00
	Question type:what is the, Accuracy 4.33
	Question type:is it, Accuracy 0.00
	Question type:which, Accuracy 9.41
	Question type:is the, Accuracy 0.00
	Question type:what does the, Accuracy 0.00
	Question type:does the, Accuracy 0.00
	Question type:what is the name, Accuracy 0.00
	Question type:why, Accuracy 0.00
	Question type:who is, Accuracy 0.00
	Question type:is he, Accuracy 0.00
	Question type:is this person, Accuracy 0.00
	Question type:is this, Accuracy 0.00
	Question type:where is the, Accuracy 0.00
	Question type:are these, Accuracy

Already finish: 12500 21
	Question type:how many, Accuracy 38.09
	Question type:how many people are, Accuracy 45.86
	Question type:what, Accuracy 5.76
	Question type:what time, Accuracy 1.16
	Question type:what number is, Accuracy 6.57
	Question type:how, Accuracy 3.30
	Question type:none of the above, Accuracy 7.97
	Question type:how many people are in, Accuracy 66.81
	Question type:what is, Accuracy 0.00
	Question type:what are the, Accuracy 0.00
	Question type:what is the, Accuracy 3.41
	Question type:is it, Accuracy 0.00
	Question type:which, Accuracy 7.27
	Question type:is the, Accuracy 0.00
	Question type:what does the, Accuracy 0.00
	Question type:does the, Accuracy 0.00
	Question type:what is the name, Accuracy 0.00
	Question type:why, Accuracy 0.00
	Question type:who is, Accuracy 0.00
	Question type:can you, Accuracy 0.00
	Question type:what brand, Accuracy 0.00
	Question type:what is on the, Accuracy 0.00
	Question type:is he, Accuracy 0.00
	Question type:is this person, Accu

Already finish: 15500 27
	Question type:how many, Accuracy 37.85
	Question type:how many people are, Accuracy 44.98
	Question type:what, Accuracy 4.84
	Question type:what time, Accuracy 1.13
	Question type:what number is, Accuracy 6.85
	Question type:how, Accuracy 3.39
	Question type:none of the above, Accuracy 7.13
	Question type:how many people are in, Accuracy 66.01
	Question type:what is, Accuracy 4.00
	Question type:what are the, Accuracy 0.00
	Question type:what is the, Accuracy 3.36
	Question type:is it, Accuracy 0.00
	Question type:which, Accuracy 8.40
	Question type:is the, Accuracy 0.00
	Question type:what does the, Accuracy 0.00
	Question type:does the, Accuracy 0.00
	Question type:what is the name, Accuracy 0.00
	Question type:why, Accuracy 0.00
	Question type:who is, Accuracy 0.00
	Question type:can you, Accuracy 0.00
	Question type:what brand, Accuracy 0.00
	Question type:what is on the, Accuracy 0.00
	Question type:is he, Accuracy 0.00
	Question type:is this person, Accu

Already finish: 18500 35
	Question type:how many, Accuracy 37.86
	Question type:how many people are, Accuracy 44.47
	Question type:what, Accuracy 4.93
	Question type:what time, Accuracy 1.07
	Question type:what number is, Accuracy 5.99
	Question type:how, Accuracy 3.26
	Question type:none of the above, Accuracy 7.82
	Question type:how many people are in, Accuracy 64.80
	Question type:what is, Accuracy 3.33
	Question type:what are the, Accuracy 0.57
	Question type:what is the, Accuracy 3.54
	Question type:is it, Accuracy 0.00
	Question type:which, Accuracy 7.78
	Question type:is the, Accuracy 0.00
	Question type:what does the, Accuracy 0.00
	Question type:does the, Accuracy 0.00
	Question type:what is this, Accuracy 0.00
	Question type:what is the name, Accuracy 0.00
	Question type:why, Accuracy 0.00
	Question type:who is, Accuracy 0.00
	Question type:can you, Accuracy 0.00
	Question type:what brand, Accuracy 0.00
	Question type:what is on the, Accuracy 0.00
	Question type:is he, Accura

Already finish: 21000 42
	Question type:how many, Accuracy 37.63
	Question type:how many people are, Accuracy 44.61
	Question type:what, Accuracy 5.47
	Question type:what time, Accuracy 1.05
	Question type:what number is, Accuracy 5.60
	Question type:how, Accuracy 3.50
	Question type:none of the above, Accuracy 7.80
	Question type:how many people are in, Accuracy 65.22
	Question type:what is, Accuracy 3.12
	Question type:what are the, Accuracy 0.50
	Question type:what is the, Accuracy 3.77
	Question type:is it, Accuracy 0.00
	Question type:which, Accuracy 7.12
	Question type:is the, Accuracy 0.00
	Question type:what does the, Accuracy 0.00
	Question type:does the, Accuracy 0.00
	Question type:what is this, Accuracy 0.00
	Question type:what is the name, Accuracy 0.00
	Question type:why, Accuracy 0.00
	Question type:who is, Accuracy 0.00
	Question type:what animal is, Accuracy 0.00
	Question type:can you, Accuracy 0.00
	Question type:what brand, Accuracy 0.00
	Question type:what is on th

Already finish: 23500 45
	Question type:how many, Accuracy 37.51
	Question type:how many people are, Accuracy 44.69
	Question type:what, Accuracy 5.25
	Question type:what time, Accuracy 1.02
	Question type:what number is, Accuracy 5.67
	Question type:how, Accuracy 3.28
	Question type:none of the above, Accuracy 7.61
	Question type:how many people are in, Accuracy 64.30
	Question type:what is, Accuracy 2.63
	Question type:what are the, Accuracy 1.38
	Question type:what is the, Accuracy 3.48
	Question type:is it, Accuracy 0.00
	Question type:which, Accuracy 6.56
	Question type:is the, Accuracy 0.00
	Question type:what does the, Accuracy 0.00
	Question type:does the, Accuracy 0.00
	Question type:what is this, Accuracy 0.00
	Question type:what is the name, Accuracy 0.00
	Question type:why, Accuracy 0.00
	Question type:who is, Accuracy 0.00
	Question type:what animal is, Accuracy 0.00
	Question type:can you, Accuracy 0.00
	Question type:what brand, Accuracy 0.00
	Question type:what is on th

Already finish: 26000 50
	Question type:how many, Accuracy 37.60
	Question type:how many people are, Accuracy 44.35
	Question type:what, Accuracy 5.38
	Question type:what time, Accuracy 0.96
	Question type:what number is, Accuracy 5.51
	Question type:how, Accuracy 3.08
	Question type:none of the above, Accuracy 7.62
	Question type:how many people are in, Accuracy 64.20
	Question type:what is, Accuracy 2.27
	Question type:what are the, Accuracy 1.27
	Question type:what is the, Accuracy 3.44
	Question type:is it, Accuracy 0.00
	Question type:which, Accuracy 6.09
	Question type:is the, Accuracy 0.00
	Question type:what does the, Accuracy 0.00
	Question type:does the, Accuracy 0.00
	Question type:what is this, Accuracy 0.00
	Question type:what is the name, Accuracy 0.00
	Question type:why, Accuracy 0.00
	Question type:who is, Accuracy 0.00
	Question type:what animal is, Accuracy 0.00
	Question type:can you, Accuracy 0.00
	Question type:what brand, Accuracy 0.00
	Question type:what is on th

Overall
	Question type:how many, Accuracy 37.64, Number of instance 20320
	Question type:how many people are, Accuracy 44.59, Number of instance 1997
	Question type:what, Accuracy 5.82, Number of instance 840
	Question type:what time, Accuracy 0.94, Number of instance 1130
	Question type:what number is, Accuracy 5.29, Number of instance 656
	Question type:how, Accuracy 3.22, Number of instance 877
	Question type:none of the above, Accuracy 7.23, Number of instance 260
	Question type:how many people are in, Accuracy 64.17, Number of instance 900
	Question type:what is, Accuracy 2.22, Number of instance 45
	Question type:what are the, Accuracy 1.17, Number of instance 77
	Question type:what is the, Accuracy 3.33, Number of instance 757
	Question type:is it, Accuracy 0.00, Number of instance 3
	Question type:which, Accuracy 5.38, Number of instance 78
	Question type:is the, Accuracy 0.00, Number of instance 7
	Question type:what does the, Accuracy 0.00, Number of instance 86
	Question typ